In [ ]:
from selenium import webdriver
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait
from src.common.WebDriver import WebDriver
from src.common.utils import *
import time
from src.instagram.Instagram import Instagram

config = loadConfiguration('config/config.yaml')
credentials = loadCredentials('config/credentials.yaml')
logger = getLogger(__name__)
logger.debug(credentials)

In [ ]:
#driver = WebDriver().getDriver()
#crawler = Crawler(credentials['instagram'], credentials['instagram']['user'])
#crawler.login()
#crawler.getFollowers()
#print(crawler.followers)
#crawler.getFollowings()
#print(crawler.followings)

In [ ]:
instagram = Instagram(credentials['instagram'], credentials['instagram']['user'])

In [ ]:
#print(instagram.followings)

In [ ]:
driver = webdriver.Chrome('drivers/chromedriver')
driver.get("https://www.instagram.com/accounts/login/")

In [ ]:
wait = WebDriverWait(driver, config['webdriver']['wait']['time'])

In [ ]:
logger.info("Logging in to instagram with user: "+credentials['instagram']['user'])
driver.get(config['instagram']['loginUrl'])

wait.until(EC.presence_of_element_located((By.XPATH, "//input[@name='username'][@type='text']")))
logger.debug("Presence of username field detected.")

userNameInput = driver.find_element_by_xpath("//input[@name='username'][@type='text']")
passwordInput = driver.find_element_by_xpath("//input[@name='password'][@type='password']")
userNameInput.send_keys(credentials['instagram']['user'])
passwordInput.send_keys(credentials['instagram']['password'])
driver.find_element_by_xpath("//button[@type='submit']").click()

In [ ]:
driver.get("https://instagram.com/"+'instafollowdev')
driver.get("https://instagram.com/"+'instafollowdev')

In [ ]:
followersButton = driver.find_element_by_xpath("//a[contains(@href, 'followers')]")
totalFollowers = int(followersButton.find_element_by_xpath("span").get_attribute("title").replace(',', ''))
logger.info("Total followers : "+str(totalFollowers))
followersButton.click()

In [ ]:
followersDialog = driver.find_element_by_xpath("//div[@role='dialog' and contains(., 'Followers')]")

In [ ]:
# Activating the popup dialog and then going to bottom of page
# wait for some-time for names to appear
try:
    element = WebDriverWait(followersDialog, config['webdriver']['wait']['time']).until(
        EC.presence_of_element_located((By.XPATH, ".//ul//li//div[text()]"))
    )
except Exception as e:
    logger.error("Error in locating name of any follower. This is done to send PAGE_DOWN down to scroll down. Error: "+repr(e))
finally:
    print("done")

followersCount = 0
previousCount = 0
scrollWithoutChangeCount = 0
scrollsWithoutChangeLimit = 10
while True:
    actions = ActionChains(driver)
    actions.send_keys_to_element(followersDialog.find_elements_by_xpath(".//ul//li//div[text()]")[-1], Keys.PAGE_DOWN) #activating scroll
    actions.perform()
    time.sleep(1)
    followersCount = len(followersDialog.find_elements_by_xpath(".//ul//li"))
    if followersCount == previousCount:
        scrollWithoutChangeCount += 1
    else:
        scrollWithoutChangeCount = 0
    if scrollWithoutChangeCount == scrollsWithoutChangeLimit:
        break
    previousCount = followersCount
    print(str(followersCount), end='\r')

In [ ]:
followers = followersDialog.find_elements_by_xpath(".//ul//li//a[@title]")
for follower in followers:
    print(follower.get_attribute('title'))

In [ ]:
closeDialogButton = driver.find_element_by_xpath("//*[@aria-label='Close']")
closeDialogButton.click()

In [ ]:
followingButton = driver.find_element_by_xpath("//a[contains(@href, 'following')]")
totalFollowing = int(followingButton.find_element_by_xpath("span").text)
logger.info("totalFollowing="+str(totalFollowing))
followingButton.click()

In [ ]:
followingDialog = driver.find_element_by_xpath("//div[@role='dialog' and contains(., 'Following')]")

In [ ]:
# wait for some-time for names to appear
try:
    firstPersonHavingName = WebDriverWait(followingDialog, config['webdriver']['wait']['time']).until(
        EC.presence_of_element_located((By.XPATH, ".//ul//li//div[text()]"))
    )
except Exception as e:
    logger.error("Error in locating name of any follower. This is done to send PAGE_DOWN down to scroll down. Error: "+repr(e))
finally:
    print("done")

followingCount = 0
while(followingCount < totalFollowing):
    actions = ActionChains(driver)
    actions.send_keys_to_element(followingDialog.find_elements_by_xpath(".//ul//li//div[text()]")[-1], Keys.PAGE_DOWN) #activating scroll
    actions.perform()
    followingCount = len(followingDialog.find_elements_by_xpath(".//ul//li"))
    print(str(followingCount))
print(str(followingCount))
